In [1]:
from dataWrangler import *
from scrape_craigslist import Scrape
from predictions import *
import urllib.request
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import requests
import datetime
import math
import glob   
import os
import io
import re

In [2]:
li=[]
for filename in glob.glob(r"..\UsedCar-PriceEstimator\data\*.csv"):
    collection = pd.read_csv(filename, index_col=None, header=0)
    li.append(collection)
frame = pd.concat(li, axis=0, ignore_index=True)

In [9]:
lis=[]
for x in range(3,29,6):
    lis.append(singlePredict('https://sfbay.craigslist.org/sby/cto/d/san-jose-excellent-2007-rx-350/7279907681.html',frame,x))

x=0
for l in lis:
    x+=(l[1])
res=int(x/len(lis))
result=((math.floor(res / 100.0)) * 100)    
    
print("-----------------------------------------------")
print(' Recommended:',result,"\n","Actual:",lis[1][-1],"\n",'Low:',min(lis)[1],'-','High:',max(lis)[1])

lexus rx Accuracy: 97.98 %.
lexus rx Accuracy: 98.31 %.
lexus rx Accuracy: 98.07 %.
lexus rx Accuracy: 98.43 %.
lexus rx Accuracy: 98.99 %.
-----------------------------------------------
 Recommended: 8600 
 Actual: 9850 
 Low: 7506 - High: 8946
